In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import glob, os
import rasterio
from datetime import datetime
import pickle
from nansat import Nansat, Domain, NSR
from tqdm import tqdm

import requests
import zipfile

%load_ext autoreload
%autoreload 2

from sea_ice_drift.lib import get_spatial_mean, get_uint8_image
from sea_ice_drift.ftlib import feature_tracking
from sea_ice_drift.pmlib import pattern_matching
from sea_ice_drift.libdefor import get_deformation_nodes
from functions import *

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import geemap
import ee
from datetime import datetime, timedelta

try:
  ee.Initialize(project = "utsa-spring2024")
except:
  ee.Authenticate()
  ee.Initialize(project = "utsa-spring2024")

## **Derive sea ice drift for the Weddell Sea**

## Sentinel-1 image from GEE

In [3]:
bound = {'north': -67, 'south': -77, 'west': -58, 'east': -43}
bound = [-59, -76, -45, -70]

In [4]:
Map = geemap.Map()
bound_ext = ee.Geometry.BBox(bound[0], bound[1], bound[2], bound[3]) #ee.Geometry.Rectangle(bbox)
Map.addLayer(bound_ext)
Map.centerObject(bound_ext, 5)
Map

Map(center=[-72.86886014135895, -51.999999999999844], controls=(WidgetControl(options=['position', 'transparen…

In [3]:
from pyproj import Transformer

def appropriate_resolution(center, pixel_size):
    # center: [lon, lat]
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3409")
    x0, y0 = transformer.transform(center[1], center[0])

    x1 = x0 + pixel_size; y1 = y0
    x2 = x0; y2 = y0 + pixel_size
    x3 = x0 + pixel_size; y3 = y0 + pixel_size
    
    x = np.array([x0, x1, x2, x3])
    y = np.array([y0, y1, y2, y3])
    
    transformer = Transformer.from_crs("EPSG:3408", "EPSG:3857")
    xc, yc = transformer.transform(x, y)
    
    d1 = ((xc[0]-xc[1])**2 + (yc[0]-yc[1])**2)**0.5
    d2 = ((xc[0]-xc[2])**2 + (yc[0]-yc[2])**2)**0.5
    d3 = ((xc[0]-xc[3])**2 + (yc[0]-yc[3])**2)**0.5
    
    dp = np.round(np.mean([d1, d2, d3]), -1)
    return dp

### Numpy direct

In [ ]:
######## START ###############################################################
year = 2021
w = [2, 0.5]
pixel_size = 60
bound = [-59, -76, -45, -70]

for clon in np.arange(bound[0], bound[2], w[0]):
    for clat in np.arange(bound[1], bound[3], w[1]):

        print(f"##### [{clon:.1f}, {clat:.1f}] #####")

        center0 = np.array([clon, clat]) # lon, lat (Actually, it's upper right corner of the polygon)
        center = center0.copy()
        bbox = [center[0]-w[0], center[1]-w[1], center[0], center[1]]
        
        extent = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]) #ee.Geometry.Rectangle(bbox)
        
        dp = appropriate_resolution(center0, pixel_size)
        
        # Define necessary functions with the defined bbox --------------------------
        def collection_addbands(img):
            bands = img.bandNames() # First band ('HH' or 'VV')
            band = [ee.Algorithms.If(bands.contains('HH'), 'HH', 'VV')]
            norm = img.select(band).divide(img.select('angle')).rename('norm')
            img2 = img.addBands(norm, overwrite=True).select('norm')
            return img2
        def add_coverage(img):    
            tol = 10000
            overlap = img.geometry().intersection(extent, tol)
            ratio = overlap.area(tol).divide(extent.area(tol))
            return img.set({'coverage_ratio': ratio})
        # calculate coverage area of image to roi
        def coverage(img):
            tol = 10000
            overlap = img.geometry().intersection(extent, tol)
            ratio = overlap.area(tol).divide(extent.area(tol))
            return ratio.getInfo()
        # -------------------------------------------------------------------------
        
        for month in range(2, 13):
        
            print(f"##### {year} {str(month).zfill(2)} #####")
        
            ### LOAD SENTINEL-1 DATA FROM GOOGLE EARTH ENGINE AND SAVE AS NUMPY ARRAYS #######################
            start_date = f"{year}-{str(month).zfill(2)}-01" #02
            if month == 12:
                end_date = f"{year+1}-01-01" #20
            else:
                end_date = f"{year}-{str(month+1).zfill(2)}-01" #20
            
            d0 = datetime.strptime(start_date, "%Y-%m-%d")
            d2 = datetime.strptime(end_date, "%Y-%m-%d")
            
            dn = abs(d2-d0).days
            
            map = geemap.Map()
            array, times, Hs, Ws = [], [], [], []
            
            for i in tqdm(range(dn)):
            
                t1 = (d0 + timedelta(days = i))
                t2 = (d0 + timedelta(days = i+1))
            
                t1_str = t1.strftime("%Y-%m-%d")
                
                collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(extent).filterDate(t1, t2)\
                .filter(ee.Filter.eq('instrumentMode', 'EW'))
                
                collection1 = collection0.map(collection_addbands)
                collection = collection1.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.7))
            
                S1_ids = collection.aggregate_array('system:id').getInfo()
            
                if len(S1_ids) > 1:
                    rs = collection.aggregate_array('coverage_ratio').getInfo()
                    S1_ids = [S1_ids[np.argmax(rs)]]
            
                for k in S1_ids:
                    
                    img = collection_addbands(ee.Image(k)).setDefaultProjection("EPSG:3857") #("EPSG:3857") #.reproject("EPSG:4326")
                    img = add_coverage(img)
            
                    name = os.path.basename(k)[17:32]
                    description = f"S1_{name}"                   
                    
                    img2 = img.clip(extent).select("norm") 
                        
                    a = geemap.ee_to_numpy(img2, scale = dp)[:, :, 0]
                    a[a>=0] = np.nan
                    array.append(a)
                    Hs.append(a.shape[0])
                    Ws.append(a.shape[1])
                    times.append(datetime.strptime(description[-15:], "%Y%m%dT%H%M%S"))

            H_target = np.bincount(Hs).argmax()
            W_target = np.bincount(Ws).argmax()
            ind_target = np.where((Hs == H_target) & (Ws == W_target))[0]

            array1 = np.zeros([len(ind_target), H_target, W_target])
            for i, n in enumerate(ind_target):
                array1[i] = array[n]
            
            del array
        
            print("SAR images: ", array1.shape[0])
            
            x0 = str(center[0].round(1))
            y0 = str(center[1].round(1))
            
            # pkl_name = f"D:\\Landfast\\numpy_image\\S1_img_{year}{str(month).zfill(2)}_{x0}_{y0}.pkl"
            # pkl_object = [array1, times, bbox]
            # with open(pkl_name, 'wb') as handle:
            #     pickle.dump(pkl_object, handle)
        
            ### DERIVE AND SAVE SEA ICE DRIFT #######################
            ind_final1 = []
            ind_final2 = []
            pkl_objects = []
            upms = []
            vpms = []
            apms = []
            rpms = []
            hpms = []
            latpms = []
            lonpms = []
            
            for ind1 in tqdm(range(0, array1.shape[0]-1)):
                ind2 = ind1 + 1
                pkl_object = derive_drift(array1[ind1], array1[ind2], times[ind1], times[ind2], pixel_size, pm_step = 30)
            
                if len(pkl_object) > 0:
                    # pkl_objects.append(pkl_object)
                    upm, vpm, apm, rpm, hpm, lon2pm, lat2pm = pkl_object
                    upms.append(upm)
                    vpms.append(vpm)
                    apms.append(apm)
                    rpms.append(rpm)
                    hpms.append(hpm)
                    latpms.append(lat2pm)
                    lonpms.append(lon2pm)
                    
                    if ind1 not in ind_final1:
                        ind_final1.append(ind1)
                    if ind2 not in ind_final2:
                        ind_final2.append(ind2)
            
            upms = np.array(upms)
            vpms = np.array(vpms)
            apms = np.array(apms)
            rpms = np.array(rpms)
            hpms = np.array(hpms)
            latpms = np.array(latpms)
            lonpms = np.array(lonpms)
            
            pkl_objects = [upms, vpms, apms, rpms, hpms, latpms, lonpms]
            
            pkl_name = f"D:\\Landfast\\numpy_image\\S1_vel_{year}{str(month).zfill(2)}_{x0}_{y0}.pkl"
            pkl_save = [pkl_objects, ind_final1, ind_final2, array1, times, bbox]
            with open(pkl_name, 'wb') as handle:
                pickle.dump(pkl_save, handle)

            print(f"Ice drift tracking: {len(ind_final1)}")
        
            ### LOAD AND SAVE ERA5 REANALYSIS DATA #######################
            days = [str(i).zfill(2) for i in range(1, 32)]
            ds, fl = retrieve_hourly_ERA5_bbox(year, str(month).zfill(2), days, bbox)
        
            era_times = ds.valid_time.values
            era_lat = ds.latitude.values
            era_lon = ds.longitude.values
            era_u10 = ds.u10.values
            era_v10 = ds.v10.values
        
            pkl_name = f"D:\\Landfast\\numpy_image\\S1_wind_{year}{str(month).zfill(2)}_{x0}_{y0}.pkl"
            pkl_save = [era_times, era_lat, era_lon, era_u10, era_v10]
            with open(pkl_name, 'wb') as handle:
                pickle.dump(pkl_save, handle)
        
            del ds
            
            try:
                os.remove(fl)
            except:
                pass
                
            print("ERA5 data is loaded!")

print("Done!")

##### [-59.0, -76.0] #####
##### 2021 02 #####


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:21<00:00,  1.28it/s]


SAR images:  17


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:32<00:00,  2.03s/it]


Ice drift tracking: 6
ERA5 data is loaded!
##### 2021 03 #####


...

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:28<00:00,  1.10it/s]


SAR images:  19


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:51<00:00,  2.88s/it]


Ice drift tracking: 17
ERA5 data is loaded!
##### 2021 08 #####


 77%|███████████████████████████████████████████████████████████████▍                  | 24/31 [14:48<07:27, 63.87s/it]

### Image download

In [ ]:
def collection_addbands(img):
    bands = img.bandNames() # First band ('HH' or 'VV')
    band = [ee.Algorithms.If(bands.contains('HH'), 'HH', 'VV')]
    norm = img.select(band).divide(img.select('angle')).rename('norm')
    img2 = img.addBands(norm, overwrite=True).select('norm')
    return img2

def add_coverage(img, extent = extent):
    
    tol = 10000
    overlap = img.geometry().intersection(extent, tol)
    ratio = overlap.area(tol).divide(extent.area(tol))
    return img.set({'coverage_ratio': ratio})

# calculate coverage area of image to roi
def coverage(img, extent):
    tol = 10000
    overlap = img.geometry().intersection(extent, tol)
    ratio = overlap.area(tol).divide(extent.area(tol))
    return ratio.getInfo()

start_date = "2018-09-01" #02
end_date = "2018-10-31" #20

d0 = datetime.strptime(start_date, "%Y-%m-%d")
d2 = datetime.strptime(end_date, "%Y-%m-%d")

dn = abs(d2-d0).days

map = geemap.Map()

for i in range(dn):

    t1 = (d0 + timedelta(days = i))
    t2 = (d0 + timedelta(days = i+1))

    t1_str = t1.strftime("%Y-%m-%d")
    
    collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(extent).filterDate(t1, t2)\
    .filter(ee.Filter.eq('instrumentMode', 'EW'))
    
    collection1 = collection0.map(collection_addbands)
    collection = collection1.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.7))

    S1_ids = collection.aggregate_array('system:id').getInfo()
        
    # print(t1, t2, len(S1_ids))

    # print(start_date, end_date, len(S1_ids))

    if len(S1_ids) > 1:
        rs = collection.aggregate_array('coverage_ratio').getInfo()
        S1_ids = [S1_ids[np.argmax(rs)]]

    for k in S1_ids:
        
        img = collection_addbands(ee.Image(k)).setDefaultProjection("EPSG:3857") #("EPSG:3857") #.reproject("EPSG:4326")
        img = add_coverage(img)

        mrange = range(0, rows)
        nrange = range(0, cols)

        out_dir = "D:\Eddies\S1_eddies"
        geemap.download_ee_image_tiles(
            img, ee.FeatureCollection(extent), out_dir, prefix=f"s1_{t1_str}_", crs="EPSG:3857", scale=dp
        )

        # array = np.zeros([len(mrange)*2400, len(nrange)*2400])
        
        r = img.get('coverage_ratio').getInfo()
        # img = collection.reduce(ee.Reducer.median()).select('norm_median').clip(region)

        name = os.path.basename(k)[17:32]
        description = f"S1_{name}"

        # a = geemap.ee_to_numpy(img, scale = dp)
        # array.append(a)
        # days.append(t1)
        
        print(description, r)
        

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s1_2018-09-01_1.tif download size (raw: 1.26 GB).


s1_2018-09-01_1.tif: |                                                | 0.00/1.26G (raw) [  0.0%] in 00:00 (et…

Downloaded 1 tiles in 113.99481678009033 seconds.
S1_20180901T075438 0.9470393927363248


Consider adjusting `region`, `scale` and/or `dtype` to reduce the s1_2018-09-02_1.tif download size (raw: 1.26 GB).


s1_2018-09-02_1.tif: |                                                | 0.00/1.26G (raw) [  0.0%] in 00:00 (et…

...

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s1_2018-10-09_1.tif download size (raw: 1.26 GB).


s1_2018-10-09_1.tif: |                                                | 0.00/1.26G (raw) [  0.0%] in 00:00 (et…

Downloaded 1 tiles in 115.09260034561157 seconds.
S1_20181009T073813 0.9918759901621361


Consider adjusting `region`, `scale` and/or `dtype` to reduce the s1_2018-10-10_1.tif download size (raw: 1.26 GB).


s1_2018-10-10_1.tif: |                                                | 0.00/1.26G (raw) [  0.0%] in 00:00 (et…

## Summarized version for bulk processing

In [4]:
files

['D:\\Eddies\\S1_eddies\\S1_20181002T074615.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181003T073726.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181004T072958.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181005T072054.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181006T080155.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181007T075439.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181008T074529.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181009T073813.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181010T072915.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181011T072144.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181012T080235.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181013T075346.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181014T074615.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181015T073726.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181016T072958.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181017T072054.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181018T080155.tif',
 'D:\\Eddies\\S1_eddies\\S1_20181019T075439.tif']

In [9]:
img = rasterio.open(files[ind1])
a1 = img.read(1)
# img.close()

In [58]:
normz = Normalize(vmin = 0.2, vmax = 0.9)

In [21]:
# normalize array values
def prepare_array(img):
    # img = (img+1)/(1)
    img -= get_spatial_mean(img)
    img = get_uint8_image(img, None, None, 1, 99)
    return img

files = sorted(glob.glob(f"D:\\Eddies\\S1_eddies\\S1_*.tif"))
drawing = True
# print(len(files))

for i in range(5, len(files)-1):

    # Image read ==============================================
    ind1 = i
    ind2 = i+1
    
    t1 = datetime.strptime(os.path.basename(files[ind1])[-19:-4], "%Y%m%dT%H%M%S")
    t2 = datetime.strptime(os.path.basename(files[ind2])[-19:-4], "%Y%m%dT%H%M%S")

    t1_str = t1.strftime("%Y%m%dT%H%M%S")
    t2_str = t2.strftime("%Y%m%dT%H%M%S")

    if t2.year >= 2010:
    
        img = rasterio.open(files[ind1])
        # print(files[ind1])
        a1 = img.read(1)
        img.close()
        
        img = rasterio.open(files[ind2])
        # print(files[ind2])
        a2 = img.read(1)
        img.close()        
        
        a1 = prepare_array(a1)
        a2 = prepare_array(a2)
        
        ds = abs(t2-t1).total_seconds()
        print("Time difference (seconds): ", ds)
    
        # Process image =====================================================
        pixel_size = 50 # meters for fake grid
        pm_step = 50 # subsampling for lon/lat grids

        srs = NSR(32631) #dstCrs = {'init': 'EPSG:32721'}
        
        rows, cols = a1.shape
    
        d = Domain(srs.wkt, f'-te 0 0 {cols*pixel_size} {rows*pixel_size} -ts {cols} {rows}')
        n1 = Nansat.from_domain(d, a1)
        n2 = Nansat.from_domain(d, a2)
        c1, r1, c2, r2 = feature_tracking(n1, n2, nFeatures=50000, ratio_test=0.5, max_drift=20000, verbose=False)
        
        if len(c1) > 10:
            # convert row/column coordinates of matched features to lon/lat
            lon1ft, lat1ft = n1.transform_points(c1, r1)
            lon2ft, lat2ft = n2.transform_points(c2, r2)
            
            # get lon/lat coordinates of the border of images
            lon1b, lat1b = n1.get_border()
            lon2b, lat2b = n2.get_border()
            
            lon1pm, lat1pm = d.get_geolocation_grids(pm_step)
            upm, vpm, apm, rpm, hpm, lon2pm, lat2pm = pattern_matching(lon1pm, lat1pm, n1, c1, r1, n2, c2, r2, srs=srs.ExportToProj4())
        
            ## Draw maps of sea ice velocity ==============================================
            # plot valid vectors in Polar stereographic projection
            gpi = (rpm > 0.4) 
            # X/Y grids in polar stereographic projection covering image_1
            x1pm, y1pm = d.get_geolocation_grids(pm_step, dst_srs=srs)
            
            # spacial reference for plotting
            crs = ccrs.UTM(zone=31, southern_hemisphere=False)

            ### SAVE AS PICKLE =======================================================
            pkl_name = f"D:\\Eddies\\ice_vel\\pickle\\S1_vel_{t1_str}_{t2_str}.pkl"
            pkl_object = [upm, vpm, apm, rpm, hpm, lon2pm, lat2pm]
            with open(pkl_name, 'wb') as handle:
                pickle.dump(pkl_object, handle)
            print(pkl_name)
            
            ### DRAW MAP of ICE VELOCITY =============================================
            if drawing:
                plt.figure(figsize=(8,8), dpi = 180)
                ax = plt.axes(projection=crs)
        
                normz = Normalize(vmin = 0.2, vmax = 0.9)
                
                ax.imshow(a1, cmap='gray', extent=[x1pm.min(), x1pm.max(), y1pm.min(), y1pm.max()], origin='upper', vmin = 100, vmax = 250)
                quiv=ax.quiver(x1pm[gpi], y1pm[gpi], upm[gpi]/ds*3600*24, vpm[gpi]/ds*3600*24, rpm[gpi],
                               cmap='jet', width=0.002, angles='xy', scale_units='xy', scale=1, norm = normz)
                plt.colorbar(quiv, shrink=0.5)
                # plt.quiverkey(quiv, x1pm.max(), y1pm.max(), 0.05, '0.05 m/s', coordinates='data')
                plt.title('Ice drfit speed [m/day]')
                # ax.add_feature(land_50m, edgecolor='black')
                ax.set_xticks(np.arange(x1pm.min(), x1pm.max(), 25000))
                ax.set_yticks(np.arange(0, y1pm.max(), 25000))
                
                ax.set_xlim([x1pm.min() - pixel_size*pm_step, x1pm.max() + pixel_size*pm_step])
                ax.set_ylim([y1pm.min() - pixel_size*pm_step, y1pm.max() + pixel_size*pm_step])
                
                # plt.show()            
                imgname = f"D:\\Eddies\\ice_vel\\maps\\S1_vel_{t1_str}_{t2_str}.png"
                print(imgname)
                plt.savefig(imgname, bbox_inches = "tight")
                plt.close()
            
print("Done")

Time difference (seconds):  85850.0


C:\Users\yoko2261\Research\sea_ice_drift\sea_ice_drift\pmlib.py:57: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


100% 04700.0 08050.0 0000nan 0000nan +0nan 0nan 0nan
 Pattern matching - OK! ( 55 sec)
D:\Eddies\ice_vel\pickle\S1_vel_20181007T075439_20181008T074529.pkl
D:\Eddies\ice_vel\maps\S1_vel_20181007T075439_20181008T074529.png
Time difference (seconds):  85964.0


C:\Users\yoko2261\Research\sea_ice_drift\sea_ice_drift\pmlib.py:57: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


100% 05850.0 07050.0 05813.0 07968.0 -03.0 0.00 0nan
 Pattern matching - OK! ( 61 sec)
D:\Eddies\ice_vel\pickle\S1_vel_20181008T074529_20181009T073813.pkl
D:\Eddies\ice_vel\maps\S1_vel_20181008T074529_20181009T073813.png
Time difference (seconds):  85862.0


...

100% 01400.0 07350.0 0000nan 0000nan +0nan 0nan 0nan
 Pattern matching - OK! ( 67 sec)
D:\Eddies\ice_vel\pickle\S1_vel_20181009T073813_20181010T072915.pkl
D:\Eddies\ice_vel\maps\S1_vel_20181009T073813_20181010T072915.png
Time difference (seconds):  85949.0


C:\Users\yoko2261\Research\sea_ice_drift\sea_ice_drift\pmlib.py:57: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


100% 06300.0 08050.0 0000nan 0000nan +0nan 0nan 0nan
 Pattern matching - OK! ( 53 sec)
D:\Eddies\ice_vel\pickle\S1_vel_20181010T072915_20181011T072144.pkl
D:\Eddies\ice_vel\maps\S1_vel_20181010T072915_20181011T072144.png
Time difference (seconds):  88851.0


C:\Users\yoko2261\Research\sea_ice_drift\sea_ice_drift\pmlib.py:57: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


100% 07150.0 07900.0 07589.0 07774.0 -03.0 0.00 0nan
 Pattern matching - OK! ( 44 sec)
D:\Eddies\ice_vel\pickle\S1_vel_20181011T072144_20181012T080235.pkl
D:\Eddies\ice_vel\maps\S1_vel_20181011T072144_20181012T080235.png
Time difference (seconds):  85871.0
Time difference (seconds):  85949.0
Time difference (seconds):  85871.0
Time difference (seconds):  85952.0
Time difference (seconds):  85856.0
Time difference (seconds):  88861.0
Time difference (seconds):  85964.0
Done
